<a href="https://colab.research.google.com/github/GA239/DS_course/blob/master/HW7/tsk1/TF2_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install tensorflow-gpu

     |████████████████████████████████| 516.2MB 33kB/s 


In [2]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf

In [3]:
DRIVE = '/content/drive/My Drive'

In [7]:
Xtrain = pd.read_csv(os.path.join(DRIVE, 'X_train.csv'), index_col=0).astype(np.float32)
Xtest = pd.read_csv(os.path.join(DRIVE, 'X_test.csv'), index_col=0).astype(np.float32)
Ytrain = pd.read_csv(os.path.join(DRIVE, 'Y_train.csv'), index_col=0).astype(np.float32)
Ytest = pd.read_csv(os.path.join(DRIVE, 'Y_test.csv'), index_col=0).astype(np.float32)

In [8]:
Xtrain.shape, Xtest.shape, Ytrain.shape, Ytest.shape 

((2967, 378), (742, 378), (2967, 3), (742, 3))

In [9]:
w = tf.Variable(tf.ones(shape=(Xtrain.shape[1], Ytrain.shape[1])), name="W")
b = tf.Variable(tf.zeros(shape=(Ytrain.shape[1])), name="b")

N_EPOCHS = 10000
display_step = 1000 
optimizer = tf.optimizers.Adam()

In [10]:
def predict(x):
    return tf.linalg.matmul(x, w) + b 

@tf.function
def lossf(y_, y):
  return tf.math.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
      labels=y, logits=y_))

@tf.function
def accf(y_, y):
  preds = tf.nn.softmax(y_)
  correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
  return tf.math.reduce_mean(tf.cast(correct_preds, tf.float32))

def calculate(x, y):
  y_ = predict(x)
  loss = lossf(y_,y)
  acc = accf(y_,y)
  return loss, acc

def forward(x, y):
  with tf.GradientTape() as t:
      loss, acc = calculate(x, y)
      gradients = t.gradient(loss, [w, b])
      optimizer.apply_gradients(zip(gradients, [w, b]))
  return loss, acc

In [11]:
print('\n--- до обучения ---\n')
_, acc_train = calculate(np.array(Xtrain), np.array(Ytrain))
print(f"Точность TRAIN: {acc_train}")
_, acc_test = calculate(np.array(Xtest), np.array(Ytest))
print(f"Точность TEST: {acc_test}")

print('\n--- обучение ---\n')
for i in range(N_EPOCHS): 
  Xbatch = Xtrain.sample(400)
  Ybatch = Ytrain.loc[Xbatch.index]
  lossv, accv = forward(np.array(Xbatch), np.array(Ybatch))
  if (i+1) % display_step == 0: 
    print(f'Эпоха {i+1}:  loss = {lossv}  ----   acc = {accv}')


print('\n--- после обучения ---\n')
_, acc_train = calculate(np.array(Xtrain), np.array(Ytrain))
print(f"Точность TRAIN: {acc_train}")
_, acc_test = calculate(np.array(Xtest), np.array(Ytest))
print(f"Точность TEST: {acc_test}")



--- до обучения ---

Точность TRAIN: 0.0532524436712265
Точность TEST: 0.04851752147078514

--- обучение ---

Эпоха 1000:  loss = 0.16117887198925018  ----   acc = 0.949999988079071
Эпоха 2000:  loss = 0.10455837100744247  ----   acc = 0.9624999761581421
Эпоха 3000:  loss = 0.05360693112015724  ----   acc = 0.9900000095367432
Эпоха 4000:  loss = 0.06060012802481651  ----   acc = 0.9750000238418579
Эпоха 5000:  loss = 0.04260117560625076  ----   acc = 0.9850000143051147
Эпоха 6000:  loss = 0.02820231206715107  ----   acc = 0.9975000023841858
Эпоха 7000:  loss = 0.037805210798978806  ----   acc = 0.987500011920929
Эпоха 8000:  loss = 0.012891344726085663  ----   acc = 0.9975000023841858
Эпоха 9000:  loss = 0.01763792894780636  ----   acc = 0.9975000023841858
Эпоха 10000:  loss = 0.015855228528380394  ----   acc = 1.0

--- после обучения ---

Точность TRAIN: 0.9949443936347961
Точность TEST: 0.9460916519165039
